In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_FXOption.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CommCCY = 'USD'
TermCCY = 'TWD'
Type = 'OPT'
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [4]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [5]:
CommZeroSP = MktSheet.cell(22, 6).value
CommZero3M = MktSheet.cell(23, 6).value
CommZero6M = MktSheet.cell(24, 6).value
CommZero1Y = MktSheet.cell(25, 6).value
print(CommZeroSP)

0.001459727696536763


In [6]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [7]:
TermZeroSP = MktSheet.cell(22, 12).value
TermZero3M = MktSheet.cell(23, 12).value
TermZero6M = MktSheet.cell(24, 12).value
TermZero1Y = MktSheet.cell(25, 12).value
print(TermZeroSP)

-0.00392904297456824


In [8]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y])
ListTermRate = list([TermZeroSP, TermZero3M, TermZero6M, TermZero1Y])
ListCommRate

[0.001459727696536763,
 0.001459727696619909,
 0.002038942990682572,
 0.002656419500050458]

In [9]:
ListTermRate

[-0.00392904297456824,
 -0.003929042973135284,
 -0.0006520395000015291,
 0.001329938573603508]

In [10]:
VecDate = ql.DateVector(ListDate)
VecCommRate = ql.DoubleVector(ListCommRate)
VecTermRate = ql.DoubleVector(ListTermRate)

In [11]:
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)
h_TSCommZero_0 = ql.YieldTermStructureHandle(TSCommZero)
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)
h_TSTermZero_0 = ql.YieldTermStructureHandle(TSTermZero)

In [12]:
NoRateCurve = len(ListCommRate)
print(NoRateCurve)

4


In [13]:
ListListCommRate = list()
ListListCommRate.append(ListCommRate)
ListListCommRate

[[0.001459727696536763,
  0.001459727696619909,
  0.002038942990682572,
  0.002656419500050458]]

In [14]:
for i in range(1, NoRateCurve):
    NewList = list()
    NewList.append(ListCommRate[0])
    for j in range(1, NoRateCurve):
        if(i==j):
            NewList.append(ListCommRate[j] + 0.0001)
        else:
            NewList.append(ListCommRate[j])
    ListListCommRate.append(NewList)

In [15]:
ListVecCommRate = list()
ListTSCommRate = list()
for i in range(NoRateCurve):
    AVecCommRate = ql.DoubleVector(ListListCommRate[i])
    ListVecCommRate.append(AVecCommRate)
    ATSCommRate = ql.ZeroCurve(VecDate, AVecCommRate, CommDC, Cal, ql.Linear(), 
                               ql.Continuous, ql.Annual)
    ListTSCommRate.append(ATSCommRate)

In [16]:
ListListTermRate = list()
ListListTermRate.append(ListTermRate)
ListListTermRate

[[-0.00392904297456824,
  -0.003929042973135284,
  -0.0006520395000015291,
  0.001329938573603508]]

In [17]:
for i in range(1, NoRateCurve):
    NewList = list()
    NewList.append(ListTermRate[0])
    for j in range(1, NoRateCurve):
        if(i==j):
            NewList.append(ListTermRate[j] + 0.0001)
        else:
            NewList.append(ListTermRate[j])
    ListListTermRate.append(NewList)

In [18]:
ListVecTermRate = list()
ListTSTermRate = list()
for i in range(NoRateCurve):
    AVecTermRate = ql.DoubleVector(ListListTermRate[i])
    ListVecTermRate.append(AVecTermRate)
    ATSTermRate = ql.ZeroCurve(VecDate, AVecTermRate, TermDC, Cal, ql.Linear(), 
                               ql.Continuous, ql.Annual)
    ListTSTermRate.append(ATSTermRate)

In [19]:
CurvUpListListCommRate = list()
CurvUpListListCommRate.append(ListCommRate)
for i in range(1, NoRateCurve):
    NewList = list()
    NewList.append(ListCommRate[0])
    for j in range(1, NoRateCurve):
        if(i==j):
            NewList.append(ListCommRate[j] + 0.017)
        else:
            NewList.append(ListCommRate[j])
    CurvUpListListCommRate.append(NewList)

In [20]:
CurvDnListListCommRate = list()
CurvDnListListCommRate.append(ListCommRate)
for i in range(1, NoRateCurve):
    NewList = list()
    NewList.append(ListCommRate[0])
    for j in range(1, NoRateCurve):
        if(i==j):
            NewList.append(ListCommRate[j] - 0.017)
        else:
            NewList.append(ListCommRate[j])
    CurvDnListListCommRate.append(NewList)

In [21]:
CurvUpListListTermRate = list()
CurvUpListListTermRate.append(ListTermRate)
for i in range(1, NoRateCurve):
    NewList = list()
    NewList.append(ListTermRate[0])
    for j in range(1, NoRateCurve):
        if(i==j):
            NewList.append(ListTermRate[j] + 0.017)
        else:
            NewList.append(ListTermRate[j])
    CurvUpListListTermRate.append(NewList)

In [22]:
CurvDnListListTermRate = list()
CurvDnListListTermRate.append(ListTermRate)
for i in range(1, NoRateCurve):
    NewList = list()
    NewList.append(ListTermRate[0])
    for j in range(1, NoRateCurve):
        if(i==j):
            NewList.append(ListTermRate[j] - 0.017)
        else:
            NewList.append(ListTermRate[j])
    CurvDnListListTermRate.append(NewList)

In [23]:
ListVolDate = list([Date6M, Date1Y])
ListStrike = list()
for i in range(9):
    ListStrike.append(MktSheet.cell(31+i, 15).value)
ListStrike

[26.75, 27, 27.25, 27.5, 27.75, 28, 28.25, 28.5, 28.75]

In [24]:
NoVolCurve = len(ListVolDate)
print(NoVolCurve)
NoStrike = len(ListStrike)
print(NoStrike)

2
9


In [25]:
VecVolDate = ql.DateVector(ListVolDate)
VecStrike = ql.DoubleVector(ListStrike)
M_Vol = ql.Matrix(9, 2, 0.0)

In [26]:
for i in range(NoStrike):
    for j in range(NoVolCurve):
        M_Vol[i][j] = MktSheet.cell(31+i, 16+j).value
print(M_Vol)   

| 0.215209 0.113231 |
| 0.138996 0.0627988 |
| 0.0627825 0.0579179 |
| 0.0535264 0.0570613 |
| 0.0524342 0.0562047 |
| 0.0527047 0.0561806 |
| 0.0542796 0.0573503 |
| 0.0558546 0.0585201 |
| 0.0868235 0.0596899 |



In [27]:
VolSurface = ql.BlackVarianceSurface(evDate, Cal, VecVolDate, VecStrike, M_Vol, DC365)
h_VolSurface = ql.BlackVolTermStructureHandle(VolSurface)

In [28]:
aVol = h_VolSurface.blackVol(0.75, 27.891)
print(aVol)

0.05500941759027123


In [29]:
OPTSheet = PstBook['Option']

In [30]:
start_row = 2
end_row = 4
start_col = 0
end_col = 11
recordcount = end_col - start_col + 1
ListOPT = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in OPTSheet[i][start_col:end_col+1]]
    ListOPT.append(row)
    print(row)

['OPT001', datetime.datetime(2021, 5, 19, 0, 0), datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2021, 9, 21, 0, 0), 'Long', 'Call', 'VLN', 'USDTWD', 'USD', 'TWD', 28, 1000000]
['OPT002', datetime.datetime(2021, 6, 13, 0, 0), datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), 'Short', 'Call', 'VLN', 'USDTWD', 'USD', 'TWD', 27.8, 1000000]
['OPT003', datetime.datetime(2021, 6, 21, 0, 0), datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), 'Long', 'Put', 'VLN', 'USDTWD', 'USD', 'TWD', 28.1, 1000000]


In [31]:
OPTBook = list()
for i in range(len(ListOPT)):
    row = ListOPT[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8])
    record.append(row[9])
    record.append(row[10])
    record.append(row[11])
    OPTBook.append(record)

In [32]:
OPTBook

[['OPT001',
  Date(19,5,2021),
  Date(21,5,2021),
  Date(21,9,2021),
  'Long',
  'Call',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  28,
  1000000],
 ['OPT002',
  Date(13,6,2021),
  Date(15,6,2021),
  Date(15,12,2021),
  'Short',
  'Call',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  27.8,
  1000000],
 ['OPT003',
  Date(21,6,2021),
  Date(23,6,2021),
  Date(23,5,2022),
  'Long',
  'Put',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  28.1,
  1000000]]

In [33]:
# Delta & Curvature
SenSheet = PstBook['GIRR']
for i in range(len(OPTBook)):
    h_TSCommZero = h_TSCommZero_0
    h_TSTermZero = h_TSTermZero_0
    
    ListValue = list()
    record = OPTBook[i]
    tradeDate = record[1]
    maturityDate = record[3]
    position = record[4]
    OptClass = record[5]
    OptType = record[6]
    VolClass = record[7]
    CommCCY = record[8]
    TermCCY = record[9]
    Strike = record[10]
    CommAmt = record[11]    
    EuExer = ql.EuropeanExercise(maturityDate)
    if (OptClass == 'Call'):
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Call, Strike)
    else:
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Put, Strike)
    
    anEuroOption = ql.EuropeanOption(VLPayoff, EuExer)
    Q_Spot = ql.SimpleQuote(FXSpot)
    hQ_Spot = ql.QuoteHandle(Q_Spot)    
    GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
    AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
    anEuroOption.setPricingEngine(AEEngine)
    if (position == 'Long'):
        Sign = 1.0
    else:
        Sign = -1.0
    
    Value = Sign * CommAmt * anEuroOption.NPV()
    ListValue.append(Value)
    print(Value)
                
    for j in range(1, NoRateCurve):
        h_TSTermZero = ql.YieldTermStructureHandle(ListTSTermRate[j])
        GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
        AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
        anEuroOption.setPricingEngine(AEEngine)    
        Value = Sign * CommAmt * anEuroOption.NPV()
        ListValue.append(Value)
        #SenSheet.cell(i+5, 1).value = record[0]
        SenSheet.cell(i+5, 40).value = ListValue[0]
        SenSheet.cell(i+5, 41).value = TermCCY

    ListDelta = list()
    for j in range(1, len(ListValue)):
        ListDelta.append((ListValue[j] - ListValue[0])*10000)
    
    for j in range(0, len(ListDelta)):    
        SenSheet.cell(i+5, j+43).value = ListDelta[j]
        
    ListVecTermRate = list()
    CurvUpListTSTermRate = list()
    for j in range(NoRateCurve):
        AVecTermRate = ql.DoubleVector(CurvUpListListTermRate[j])
        ListVecTermRate.append(AVecTermRate)
        ATSTermRate = ql.ZeroCurve(VecDate, AVecTermRate, TermDC, Cal, ql.Linear(), 
                                   ql.Continuous, ql.Annual)
        CurvUpListTSTermRate.append(ATSTermRate)
                
    ListVecTermRate = list()
    CurvDnListTSTermRate = list()
    for j in range(NoRateCurve):
        AVecTermRate = ql.DoubleVector(CurvDnListListTermRate[j])
        ListVecTermRate.append(AVecTermRate)
        ATSTermRate = ql.ZeroCurve(VecDate, AVecTermRate, TermDC, Cal, ql.Linear(), 
                                   ql.Continuous, ql.Annual)
        CurvDnListTSTermRate.append(ATSTermRate)
        
    CurvUpListHTSTermRate = list()
    for j in range(NoRateCurve):
        CurvUpListHTSTermRate.append(ql.YieldTermStructureHandle(CurvUpListTSTermRate[j]))
        
    CurvDnListHTSTermRate = list()
    for j in range(NoRateCurve):
        CurvDnListHTSTermRate.append(ql.YieldTermStructureHandle(CurvDnListTSTermRate[j]))
        
    CurvUpValue = list()
    for j in range(NoRateCurve):
        h_TSTermZero = CurvUpListHTSTermRate[j]
        GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
        AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
        anEuroOption.setPricingEngine(AEEngine)    
        Value = Sign * CommAmt * anEuroOption.NPV()
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoRateCurve):
        h_TSTermZero = CurvDnListHTSTermRate[j]
        GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
        AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
        anEuroOption.setPricingEngine(AEEngine)    
        Value = Sign * CommAmt * anEuroOption.NPV()            
        CurvDnValue.append(Value)        
        
    CurvatureUp = list()
    ShiftUpValue = list()
    DeltaChange = list()
    for j in range(1, NoRateCurve):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - 0.017 * ListDelta[j-1]))
                
    for j in range(0, len(CurvatureUp)):    
        SenSheet.cell(i+5, j+55).value = CurvatureUp[j]
            
    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoRateCurve):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])    
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + 0.017 * ListDelta[j-1]))

    for j in range(0, len(CurvatureDn)):    
        SenSheet.cell(i+5, j+67).value = CurvatureDn[j]
        
PstBook.save('Pst_FXOption.xlsx')
PstBook.close()

214380.82208287125
-421230.21349021135
727194.1754706659
